In [5]:
import pandas as pd
import numpy as np

# 文件路径
excel_path = 'Testing cases for temperature condition_new.xlsx'
xls = pd.ExcelFile(excel_path)

# 期望的列顺序
expected_cols = ['Time (s)', 'T_outer (C)', 'T_inner (C)', 'T_ave (C)']

for sheet in xls.sheet_names:
    print(f"\n处理 sheet: {sheet}")
    df = pd.read_excel(excel_path, sheet_name=sheet, header=None)

    found = False

    # 寻找 "Simulation result" 区域的 "Time (s)"
    for row_idx in range(df.shape[0]):
        for col_idx in range(df.shape[1]):
            cell = str(df.iat[row_idx, col_idx]).strip()
            if cell == 'Time (s)' and col_idx >= 3:  # 确保是右侧的 Time (s)
                print(f"找到 Time (s) 在位置: 行{row_idx}, 列{col_idx}")

                # 检查是否有足够的列
                if col_idx + 3 >= df.shape[1]:
                    print(f"警告：列数不足，需要4列但只有{df.shape[1] - col_idx}列可用")
                    continue

                # 提取数据区域（包含标题行）
                data_start_row = row_idx
                data_end_col = min(col_idx + 4, df.shape[1])  # 确保不超出边界

                # 提取子数据框
                sub_df = df.iloc[data_start_row:, col_idx:data_end_col].copy()

                # 设置列名
                if sub_df.shape[1] == 4:
                    sub_df.columns = expected_cols
                else:
                    print(f"警告：列数不匹配，期望4列，实际{sub_df.shape[1]}列")
                    continue

                # 去掉标题行
                sub_df = sub_df.iloc[1:].copy()

                # 清理数据
                sub_df = sub_df.replace('', np.nan)  # 空字符串转为NaN
                sub_df = sub_df.dropna(how='all')    # 去掉全空行

                # 确保数值列为数值类型
                for col in expected_cols:
                    if col in sub_df.columns:
                        sub_df[col] = pd.to_numeric(sub_df[col], errors='coerce')

                # 去掉全为NaN的行
                sub_df = sub_df.dropna(how='all')

                # 保存到CSV
                output_filename = f'(test){sheet}.csv'
                sub_df.to_csv(output_filename, index=False)

                print(f'✅ 提取成功：{sheet} → {output_filename}')
                print(f"   数据形状: {sub_df.shape}")
                print(f"   前几行预览:")
                print(sub_df.head())

                found = True
                break
        if found:
            break

    if not found:
        print(f'❌ 在 sheet {sheet} 中找不到合适的 "Time (s)" (Simulation result区域)')

print("\n处理完成！")


处理 sheet: Case (1)
找到 Time (s) 在位置: 行1, 列3
✅ 提取成功：Case (1) → (test)Case (1).csv
   数据形状: (100, 4)
   前几行预览:
   Time (s)  T_outer (C)  T_inner (C)  T_ave (C)
2       150       195.00       196.20     195.11
3       290       195.00       197.35     195.27
4       440       195.00       198.58     195.49
5       580       195.00       199.74     195.72
6       720       195.01       200.89     195.98

处理 sheet: Case (2)
找到 Time (s) 在位置: 行1, 列3
✅ 提取成功：Case (2) → (test)Case (2).csv
   数据形状: (100, 4)
   前几行预览:
   Time (s)  T_outer (C)  T_inner (C)  T_ave (C)
2       150       285.00       286.53     285.13
3       290       285.00       288.00     285.33
4       440       285.00       289.57     285.60
5       580       285.00       291.04     285.88
6       720       285.01       292.51     286.20

处理 sheet: Case (3)
找到 Time (s) 在位置: 行1, 列3
✅ 提取成功：Case (3) → (test)Case (3).csv
   数据形状: (100, 4)
   前几行预览:
   Time (s)  T_outer (C)  T_inner (C)  T_ave (C)
2       170       208.00       208.5

In [10]:
import pandas as pd
import os
import glob

def add_initial_condition_simple(csv_file):
    """
    为单个CSV文件添加初始条件（简化版）
    """
    try:
        # 读取CSV
        df = pd.read_csv(csv_file)

        # 检查列名
        required_cols = ['Time (s)', 'T_outer (C)', 'T_inner (C)', 'T_ave (C)']
        if not all(col in df.columns for col in required_cols):
            print(f"跳过文件 {csv_file}：缺少必需的列")
            return False

        # 检查是否已有t=0
        if len(df) > 0 and df['Time (s)'].iloc[0] == 0:
            print(f"文件 {os.path.basename(csv_file)} 已有初始条件")
            return True
        initial_temp = df['T_outer (C)'].iloc[1]
        print(initial_temp)
        # 添加初始行
        initial_row = {
            'Time (s)': 0,
            'T_outer (C)': initial_temp,
            'T_inner (C)': initial_temp,
            'T_ave (C)': initial_temp
        }

        # 在开头插入初始行
        df_new = pd.concat([pd.DataFrame([initial_row]), df], ignore_index=True)

        # 保存
        df_new.to_csv(csv_file, index=False)
        print(f"✓ 已添加初始条件到 {os.path.basename(csv_file)}")
        return True

    except Exception as e:
        print(f"✗ 处理 {csv_file} 时出错: {e}")
        return False

# 快速处理所有CSV文件
def process_all_csvs():
    # 查找当前目录及子目录中的所有CSV文件
    csv_files = glob.glob("**/*.csv", recursive=True)

    if not csv_files:
        print("未找到CSV文件")
        return

    print(f"找到 {len(csv_files)} 个CSV文件")

    success_count = 0
    for csv_file in csv_files:
        if add_initial_condition_simple(csv_file):
            success_count += 1

    print(f"\n完成！成功处理 {success_count}/{len(csv_files)} 个文件")

if __name__ == "__main__":
    process_all_csvs()

找到 217 个CSV文件
210.0
✓ 已添加初始条件到 (test)Case (36).csv
118.0
✓ 已添加初始条件到 (test)Case (20).csv
144.0
✓ 已添加初始条件到 (test)Case (16).csv
235.0
✓ 已添加初始条件到 (test)Case (17).csv
217.18
✓ 已添加初始条件到 (test)Case (40).csv
195.0
✓ 已添加初始条件到 (test)Case (1).csv
188.0
✓ 已添加初始条件到 (test)Case (21).csv
150.0
✓ 已添加初始条件到 (test)Case (37).csv
285.0
✓ 已添加初始条件到 (test)Case (10).csv
164.41
✓ 已添加初始条件到 (test)Case (30).csv
109.01
✓ 已添加初始条件到 (test)Case (26).csv
100.0
✓ 已添加初始条件到 (test)Case (6).csv
100.0
✓ 已添加初始条件到 (test)Case (7).csv
220.01
✓ 已添加初始条件到 (test)Case (27).csv
215.58
✓ 已添加初始条件到 (test)Case (31).csv
165.0
✓ 已添加初始条件到 (test)Case (11).csv
155.0
✓ 已添加初始条件到 (test)Case (12).csv
265.0
✓ 已添加初始条件到 (test)Case (8).csv
255.01
✓ 已添加初始条件到 (test)Case (28).csv
411.0
✓ 已添加初始条件到 (test)Case (4).csv
100.0
✓ 已添加初始条件到 (test)Case (24).csv
125.0
✓ 已添加初始条件到 (test)Case (32).csv
205.0
✓ 已添加初始条件到 (test)Case (33).csv
188.0
✓ 已添加初始条件到 (test)Case (25).csv
100.0
✓ 已添加初始条件到 (test)Case (5).csv
167.04
✓ 已添加初始条件到 (test)Case (29).csv
205.0
✓ 已添加初始条件到 (test)